In [1]:
import pandas as pd

In [2]:
# setup plotting
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()

In [40]:
data = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv",
                   parse_dates=['date'],
                   index_col=['state', 'county', 'date'])

data = data.sort_index()  # avoid PerformanceWarning:indexing past lexsort depth may impact performance.
data

fips  cases  deaths
state   county   date                              
Alabama Autauga  2020-03-24   1001.0      1       0
                 2020-03-25   1001.0      4       0
                 2020-03-26   1001.0      6       0
                 2020-03-27   1001.0      6       0
                 2020-03-28   1001.0      6       0
...                              ...    ...     ...
Wyoming Teton    2020-03-30  56039.0     17       0
        Washakie 2020-03-27  56043.0      1       0
                 2020-03-28  56043.0      1       0
                 2020-03-29  56043.0      1       0
                 2020-03-30  56043.0      1       0

[21799 rows x 3 columns]

In [114]:
# New Cases: Difference between current day and prior day total reported cases
data['new cases'] = data.groupby(['state', 'county'])['cases'].diff()

In [115]:
# Recovered Cases: Assuming 14 day recovery period, total cases reported 14 days prior less any deaths.
recovery_period = 14  # days
shifted_cases = data.groupby(['state', 'county'])['cases'].shift(recovery_period)
#shifted_cases
data['recovered cases'] = data.apply(lambda row: shifted_cases[row.name] - row['deaths'], axis=1)

In [119]:
# Active Cases: Difference between current day total reported cases less reported deaths and recovered cases.
data['active cases'] = data.apply(lambda row: row['cases'] - row['deaths'] - row['recovered cases'], axis=1)

In [126]:
state = 'Texas'
county = 'Travis'

In [127]:
county_data = data.loc[(state, county)][['cases', 'new cases', 'deaths', 'recovered cases', 'active cases']]
county_data

,cases,new cases,deaths,recovered cases,active cases
date,,,,,
2020-03-13,4,NaN,0,NaN,NaN
2020-03-14,4,0.0,0,NaN,NaN
2020-03-15,6,2.0,0,NaN,NaN
2020-03-16,6,0.0,0,NaN,NaN
2020-03-17,6,0.0,0,NaN,NaN
2020-03-18,6,0.0,0,NaN,NaN
2020-03-19,7,1.0,0,NaN,NaN
2020-03-20,21,14.0,0,NaN,NaN
2020-03-21,62,41.0,0,NaN,NaN


In [128]:
county_data.iplot(title=f"{county} County, {state}")